In [1]:
import os
import time
import math
import magenta.music as mm
import tensorflow as tf
from magenta.models.performance_rnn import performance_sequence_generator
from magenta.models.shared import sequence_generator_bundle
from magenta.music import DEFAULT_QUARTERS_PER_MINUTE
from magenta.music.protobuf.generator_pb2 import GeneratorOptions
from magenta.music.protobuf.music_pb2 import NoteSequence
from visual_midi import Plotter

print(tf.test.gpu_device_name())

/device:GPU:0


In [2]:
def generate(bundle_name, seq_generator, gen_id, primer, qpm = DEFAULT_QUARTERS_PER_MINUTE,
             notes_per_second = None, pitch_class_histogram = None, total_steps = 64, 
             temp = 1.0, beam_size = 1, branch_factor = 1, steps_per_iteration = 1):
    
    # Download bundle
    mm.notebook_utils.download_bundle(bundle_name, 'bundles')
    bundle = sequence_generator_bundle.read_bundle_file(os.path.join("bundles", bundle_name))
    
    # Initialize generator
    generator_map = seq_generator.get_generator_map()
    generator = generator_map[gen_id](checkpoint = None, bundle = bundle)
    generator.initialize()
    
    # Get primer sequence
    if primer:
        primer_seq = mm.midi_io.midi_file_to_note_sequence(os.path.join('primers', primer))
    else:
        primer_seq = NoteSequence()
        
    # Get QPM (Quarter-Notes per Minute)
    if primer_seq.tempos:
        if len(primer_seq.tempos) > 1:
            raise Exception('No support for multiple tempos')
        qpm = primer_seq.tempos[0].qpm
        
    secs_per_step = 60.0 / (qpm * getattr(generator, 'steps_per_quarter', 4))
    primer_seq_steps = math.ceil(primer_seq.total_time / secs_per_step)
    primer_seq_time = primer_seq_steps * secs_per_step
    primer_end_ad = (0.00001 if primer_seq_time > 0 else 0)
    primer_start_time = 0
    primer_end_time = primer_start_time + primer_seq_time - primer_end_ad
    
    gen_steps = total_steps - primer_seq_steps
    
    if gen_steps <= 0:
        raise Exception('Total steps too small : {}. Need at least 1 bar bigger than primer : {}'.format(
        total_steps, primer_seq_steps))
    
    gen_time = gen_steps * secs_per_step
    gen_start_time = primer_end_time
    gen_end_time = gen_start_time + gen_time + primer_end_ad
    
    print('Primer start : {} \t Primer end : {}'.format(primer_start_time, primer_end_time))
    print('Generation start : {} \t Generation end : {}'.format(gen_start_time, gen_end_time))
    
    generator_options = GeneratorOptions()
    generator_options.args['temperature'].float_value = temp
    generator_options.args['beach_size'].int_value = beam_size
    generator_options.args['branch_factor'].int_value = branch_factor
    generator_options.args['steps_per_iteration'].int_value = steps_per_iteration
    if notes_per_second:
        generator_options.args['notes_per_second'].string_value = notes_per_second
    if pitch_class_histogram:
        generator_options.args['pitch_class_histogram'].string_value = pitch_class_histogram
    generator_options.generate_sections.add(start_time = gen_start_time, end_time = gen_end_time)
    
    seq = generator.generate(primer_seq, generator_options)
    
    # Write midi file
    date_time = time.strftime('%Y-%m-%d_%H%M%S')
    gen_name = str(generator.__class__).split('.')[2]
    midi = '%s_%s_%s.mid' % (gen_name, gen_id, date_time)
    midi_path = os.path.join('output', midi)
    mm.midi_io.note_sequence_to_midi_file(seq, midi_path)
    print('Generated midi : {}'.format(os.path.abspath(midi_path)))
    
    # Generate plot
    date_time = time.strftime('%Y-%m-%d_%H%M%S')
    gen_name = str(generator.__class__).split('.')[2]
    plot = '%s_%s_%s.html' % (gen_name, gen_id, date_time)
    plot_path = os.path.join('output', plot)
    
    pretty_midi = mm.midi_io.note_sequence_to_pretty_midi(seq)
    plotter = Plotter()
    plotter.show(pretty_midi, plot_path)
    print('Generated plot : {}'.format(os.path.abspath(plot_path)))
    
    return seq

In [3]:
generate('performance_with_dynamics.mag', performance_sequence_generator, 'performance_with_dynamics',
         primer = 'Game_of_Thrones_Melody_Polyphonic.mid')

'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from C:\Users\siddh\AppData\Local\Temp\tmp0mn4sfu4\model.ckpt
Primer start : 0 	 Primer end : 4.235282000000001
Generation start : 4.235282000000001 	 Generation end : 11.294112
INFO:tensorflow:Need to generate 705 more steps for this sequence, will try asking for 423 RNN steps
INFO:tensorflow:Beam search yields sequence with log-likelihood: -1224.652954 
Generated midi : C:\Users\siddh\Documents\GitHub\magenta_project\output\performance_rnn_performance_with_dynamics_2020-12-05_180130.mid
Generated plot : C:\Users\siddh\Documents\GitHub\magenta_project\output\performance_rnn_performance_with_dynamics_2020-12-05_180130.html


ticks_per_quarter: 220
notes {
  pitch: 60
  velocity: 77
  end_time: 0.33
}
notes {
  pitch: 63
  velocity: 77
  end_time: 0.33
}
notes {
  pitch: 68
  velocity: 77
  end_time: 0.33
}
notes {
  pitch: 72
  velocity: 77
  end_time: 0.33
}
notes {
  pitch: 63
  velocity: 77
  start_time: 0.35000000000000003
  end_time: 0.6900000000000001
}
notes {
  pitch: 68
  velocity: 77
  start_time: 0.71
  end_time: 0.87
}
notes {
  pitch: 70
  velocity: 77
  start_time: 0.88
  end_time: 1.05
}
notes {
  pitch: 72
  velocity: 77
  start_time: 1.06
  end_time: 1.3900000000000001
}
notes {
  pitch: 63
  velocity: 77
  start_time: 1.41
  end_time: 1.75
}
notes {
  pitch: 68
  velocity: 77
  start_time: 1.76
  end_time: 1.93
}
notes {
  pitch: 70
  velocity: 77
  start_time: 1.94
  end_time: 2.11
}
notes {
  pitch: 63
  velocity: 77
  start_time: 2.12
  end_time: 2.45
}
notes {
  pitch: 67
  velocity: 77
  start_time: 2.12
  end_time: 2.45
}
notes {
  pitch: 70
  velocity: 77
  start_time: 2.12
  end_t

In [13]:
generate('performance_with_dynamics.mag', performance_sequence_generator, 'performance_with_dynamics', 
         primer = 'Game_of_Thrones_Melody_Polyphonic.mid')

'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.


'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.


INFO:tensorflow:Restoring parameters from C:\Users\siddh\AppData\Local\Temp\tmp_z23uzu7\model.ckpt


INFO:tensorflow:Restoring parameters from C:\Users\siddh\AppData\Local\Temp\tmp_z23uzu7\model.ckpt


Primer start : 0 	 Primer end : 4.235282000000001
Generation start : 4.235282000000001 	 Generation end : 11.294112
INFO:tensorflow:Need to generate 705 more steps for this sequence, will try asking for 423 RNN steps


INFO:tensorflow:Need to generate 705 more steps for this sequence, will try asking for 423 RNN steps


INFO:tensorflow:Beam search yields sequence with log-likelihood: -1042.763672 


INFO:tensorflow:Beam search yields sequence with log-likelihood: -1042.763672 


Generated midi : C:\Users\siddh\Documents\GitHub\magenta_project\output\performance_rnn_performance_with_dynamics_2020-12-05_072108.mid
Generated plot : C:\Users\siddh\Documents\GitHub\magenta_project\output\performance_rnn_performance_with_dynamics_2020-12-05_072108.html


ticks_per_quarter: 220
notes {
  pitch: 60
  velocity: 77
  end_time: 0.33
}
notes {
  pitch: 63
  velocity: 77
  end_time: 0.33
}
notes {
  pitch: 68
  velocity: 77
  end_time: 0.33
}
notes {
  pitch: 72
  velocity: 77
  end_time: 0.33
}
notes {
  pitch: 63
  velocity: 77
  start_time: 0.35000000000000003
  end_time: 0.6900000000000001
}
notes {
  pitch: 68
  velocity: 77
  start_time: 0.71
  end_time: 0.87
}
notes {
  pitch: 70
  velocity: 77
  start_time: 0.88
  end_time: 1.05
}
notes {
  pitch: 72
  velocity: 77
  start_time: 1.06
  end_time: 1.3900000000000001
}
notes {
  pitch: 63
  velocity: 77
  start_time: 1.41
  end_time: 1.75
}
notes {
  pitch: 68
  velocity: 77
  start_time: 1.76
  end_time: 1.93
}
notes {
  pitch: 70
  velocity: 77
  start_time: 1.94
  end_time: 2.11
}
notes {
  pitch: 63
  velocity: 77
  start_time: 2.12
  end_time: 2.45
}
notes {
  pitch: 67
  velocity: 77
  start_time: 2.12
  end_time: 2.45
}
notes {
  pitch: 70
  velocity: 77
  start_time: 2.12
  end_t

In [20]:
generate("pitch_conditioned_performance_with_dynamics.mag", performance_sequence_generator,
         "pitch_conditioned_performance_with_dynamics", primer = "Game_of_Thrones_Melody_Polyphonic.mid",
         pitch_class_histogram = "[1, 0, 1, 0, 1, 2, 0, 1, 0, 1, 0, 1]")

INFO:tensorflow:Restoring parameters from C:\Users\siddh\AppData\Local\Temp\tmpy9xu_0dt\model.ckpt


INFO:tensorflow:Restoring parameters from C:\Users\siddh\AppData\Local\Temp\tmpy9xu_0dt\model.ckpt


Primer start : 0 	 Primer end : 4.235282000000001
Generation start : 4.235282000000001 	 Generation end : 11.294112
INFO:tensorflow:Need to generate 705 more steps for this sequence, will try asking for 423 RNN steps


INFO:tensorflow:Need to generate 705 more steps for this sequence, will try asking for 423 RNN steps


INFO:tensorflow:Beam search yields sequence with log-likelihood: -1220.331421 


INFO:tensorflow:Beam search yields sequence with log-likelihood: -1220.331421 


Generated midi : C:\Users\siddh\Documents\GitHub\magenta_project\output\performance_rnn_pitch_conditioned_performance_with_dynamics_2020-12-05_072526.mid
Generated plot : C:\Users\siddh\Documents\GitHub\magenta_project\output\performance_rnn_pitch_conditioned_performance_with_dynamics_2020-12-05_072526.html


ticks_per_quarter: 220
notes {
  pitch: 60
  velocity: 77
  end_time: 0.33
}
notes {
  pitch: 63
  velocity: 77
  end_time: 0.33
}
notes {
  pitch: 68
  velocity: 77
  end_time: 0.33
}
notes {
  pitch: 72
  velocity: 77
  end_time: 0.33
}
notes {
  pitch: 63
  velocity: 77
  start_time: 0.35000000000000003
  end_time: 0.6900000000000001
}
notes {
  pitch: 68
  velocity: 77
  start_time: 0.71
  end_time: 0.87
}
notes {
  pitch: 70
  velocity: 77
  start_time: 0.88
  end_time: 1.05
}
notes {
  pitch: 72
  velocity: 77
  start_time: 1.06
  end_time: 1.3900000000000001
}
notes {
  pitch: 63
  velocity: 77
  start_time: 1.41
  end_time: 1.75
}
notes {
  pitch: 68
  velocity: 77
  start_time: 1.76
  end_time: 1.93
}
notes {
  pitch: 70
  velocity: 77
  start_time: 1.94
  end_time: 2.11
}
notes {
  pitch: 63
  velocity: 77
  start_time: 2.12
  end_time: 2.45
}
notes {
  pitch: 67
  velocity: 77
  start_time: 2.12
  end_time: 2.45
}
notes {
  pitch: 70
  velocity: 77
  start_time: 2.12
  end_t